In [ ]:
import pandas as pd               
import os                         
from sklearn import preprocessing 
import time                       

seconds = time.time()            

print("This process may take 5 to 10 minutes, depending on the performance of your computer.\n\n\n")

number = "0123456789"            # cifre ca sa detectam liniile valide in CSv-uri


csv_files = [
    "Monday-WorkingHours.pcap_ISCX",
    "Tuesday-WorkingHours.pcap_ISCX",
    "Wednesday-workingHours.pcap_ISCX",
    "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX",
    "Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX",
    "Friday-WorkingHours-Morning.pcap_ISCX",
    "Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX",
    "Friday-WorkingHours-Afternoon-DDos.pcap_ISCX",
]

main_labels=["Flow ID","Source IP","Source Port","Destination IP","Destination Port","Protocol","Timestamp","Flow Duration","Total Fwd Packets",
   "Total Backward Packets","Total Length of Fwd Packets","Total Length of Bwd Packets","Fwd Packet Length Max","Fwd Packet Length Min",
   "Fwd Packet Length Mean","Fwd Packet Length Std","Bwd Packet Length Max","Bwd Packet Length Min","Bwd Packet Length Mean","Bwd Packet Length Std",
   "Flow Bytes/s","Flow Packets/s","Flow IAT Mean","Flow IAT Std","Flow IAT Max","Flow IAT Min","Fwd IAT Total","Fwd IAT Mean","Fwd IAT Std","Fwd IAT Max",
   "Fwd IAT Min","Bwd IAT Total","Bwd IAT Mean","Bwd IAT Std","Bwd IAT Max","Bwd IAT Min","Fwd PSH Flags","Bwd PSH Flags","Fwd URG Flags","Bwd URG Flags",
   "Fwd Header Length","Bwd Header Length","Fwd Packets/s","Bwd Packets/s","Min Packet Length","Max Packet Length","Packet Length Mean","Packet Length Std",
   "Packet Length Variance","FIN Flag Count","SYN Flag Count","RST Flag Count","PSH Flag Count","ACK Flag Count","URG Flag Count","CWE Flag Count",
   "ECE Flag Count","Down/Up Ratio","Average Packet Size","Avg Fwd Segment Size","Avg Bwd Segment Size","faulty-Fwd Header Length","Fwd Avg Bytes/Bulk",
   "Fwd Avg Packets/Bulk","Fwd Avg Bulk Rate","Bwd Avg Bytes/Bulk","Bwd Avg Packets/Bulk","Bwd Avg Bulk Rate","Subflow Fwd Packets","Subflow Fwd Bytes",
   "Subflow Bwd Packets","Subflow Bwd Bytes","Init_Win_bytes_forward","Init_Win_bytes_backward","act_data_pkt_fwd",
   "min_seg_size_forward","Active Mean","Active Std","Active Max","Active Min","Idle Mean","Idle Std","Idle Max","Idle Min","Label","External IP"]

main_labels2 = main_labels              #copie header original pt referinte
main_labels = ",".join(i for i in main_labels) + "\n"  # covertim lista intr-un string csv

flag = True  # Flag sa scriem header o singura data in fisierul alldata

# parcurgere fiecare csv
for i in range(len(csv_files)):
    ths = open(str(i)+".csv", "w")     # fisier temporar cu numele csv-ului curent deschis
    ths.write(main_labels)              # Scriem headerul corect in acest fișier
    
    # Deschidem CSV original din folderul CSVs
    with open("./CSVs/"+csv_files[i]+".csv", "r") as file:
        while True:
            try:
                line = file.readline()
                if not line:
                    break
                if line[0] in number:  # Dacă linia incepe cu cifra este valida
                    # Inlocuim caracterul unicode „–” cu cratima normala „-”
                    if " – " in str(line):
                        line = str(line).replace(" – "," - ")
                    # Inlocuim valorile invalide cu zero
                    line = str(line).replace("inf","0")
                    line = str(line).replace("Infinity","0")
                    line = str(line).replace("NaN","0")
                    ths.write(str(line))  # scriem linia curata
                else:
                    continue
            except:
                break
    ths.close()

    #  citim fisierul curat in dataframe
    df = pd.read_csv(str(i)+".csv", low_memory=False)
    df = df.fillna(0)  # tot ce e lipsa punem cu val 0

    # coloane care pto avea valori nenumerice string
    string_features = ["Flow Bytes/s","Flow Packets/s"]
    for ii in string_features:
        df[ii] = df[ii].replace('Infinity', 0)
        df[ii] = df[ii].replace('NaN', 0)
        number_or_not = []
        for iii in df[ii]:
            try:
                k = int(float(iii))
                number_or_not.append(int(k))
            except:
                number_or_not.append(iii)  # daca nu merge convertit lasam val originala
        df[ii] = number_or_not

    # gasim coloanele object (string/categoric)
    string_features = []
    for j in main_labels2:
        if df[j].dtype == "object":
            string_features.append(j)

    # nu codificam Label, il lasam asa cum este
    try:
        string_features.remove('Label')
    except:
        print("error!")

    # Convertim coloanele string in numerice cu LabelEncoder
    labelencoder_X = preprocessing.LabelEncoder()
    for ii in string_features:
        try:
            df[ii] = labelencoder_X.fit_transform(df[ii])
        except:
            df[ii] = df[ii].replace('Infinity', -1)

    # Stergem coloana 61: "faulty-Fwd Header Length" (duplicata)
    df = df.drop(main_labels2[61], axis=1)

    # punem continutul la dataframe in fisierul final
    if flag: #scriem header doar o singura data
        df.to_csv('all_data.csv', index=False)
        flag = False
    else:
        df.to_csv('all_data.csv', index=False, header=False, mode="a")

    # stergere fisier temporar
    os.remove(str(i)+".csv")
    print("The pre-processing phase of the ", csv_files[i], " file is completed.\n")

print("Total operation time: = ", time.time()- seconds ,"seconds")